In [1]:
!pip install torch torchvision numpy scikit-learn pillow tqdm

In [2]:
import os
import copy
import time
import torch
import torchvision
import numpy as np
from torch import nn, optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import kagglehub

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Download the latest version of the dataset
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/deepfake-and-real-images


In [5]:
dataset_dir = os.path.join(path, "Dataset")

In [6]:
# Define transforms for training, validation, and testing (no augmentation)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]),
}

In [7]:
# Define directories for each split
train_dir = os.path.join(dataset_dir, "Train")
val_dir   = os.path.join(dataset_dir, "Validation")
test_dir  = os.path.join(dataset_dir, "Test")

In [8]:
# Load datasets using ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset   = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset  = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

In [9]:
# Create dataloaders
batch_size = 32
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
}

In [10]:
# Print Dataset Insights

def print_dataset_insights(dataset, split_name):
    class_counts = {cls: 0 for cls in dataset.classes}
    for _, label in dataset.samples:
        class_counts[dataset.classes[label]] += 1
    print(f"\n{split_name} Dataset Insights:")
    print(f"Total samples: {len(dataset)}")
    for cls, count in class_counts.items():
        print(f"  Class '{cls}': {count} images")

print("Classes:", train_dataset.classes)
print_dataset_insights(train_dataset, "Train")
print_dataset_insights(val_dataset, "Validation")
print_dataset_insights(test_dataset, "Test")

Classes: ['Fake', 'Real']

Train Dataset Insights:
Total samples: 140002
  Class 'Fake': 70001 images
  Class 'Real': 70001 images

Validation Dataset Insights:
Total samples: 39428
  Class 'Fake': 19641 images
  Class 'Real': 19787 images

Test Dataset Insights:
Total samples: 10905
  Class 'Fake': 5492 images
  Class 'Real': 5413 images


In [11]:
# Use a pretrained ResNet18 from torchvision and adjust the final layer for binary classification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# For binary classification, we output 1 logit (BCEWithLogitsLoss applies sigmoid internally)
model.fc = nn.Linear(num_ftrs, 1)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
    
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 82.3MB/s]


Using 2 GPUs!


In [12]:
# Define Loss Function, Optimizer, and Scheduler (Callbacks)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5,
                                                 patience=2, min_lr=1e-7, verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [13]:
# Training Loop with Checkpointing and Early Stopping

num_epochs = 20
early_stop_patience = 3
best_val_auc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
epochs_no_improve = 0

In [14]:
for epoch in range(1, num_epochs+1):
    print(f"\nEpoch {epoch}/{num_epochs}")

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        all_labels = []
        all_preds = []

        pbar = tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Phase", leave=False)
        for inputs, labels in pbar:
            inputs = inputs.to(device)
            labels = labels.to(device).float().unsqueeze(1)  # For BCEWithLogitsLoss
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                preds = torch.sigmoid(outputs).detach().cpu().numpy()
                all_preds.extend(preds.flatten().tolist())
                all_labels.extend(labels.cpu().numpy().flatten().tolist())

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            pbar.set_postfix({"Loss": f"{loss.item():.4f}"})

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        try:
            epoch_auc = roc_auc_score(all_labels, all_preds)
        except ValueError:
            epoch_auc = 0.0

        all_preds_labels = (np.array(all_preds) >= 0.5).astype(int)
        all_labels_np = np.array(all_labels).astype(int)
        epoch_accuracy = (all_preds_labels == all_labels_np).mean()
        current_lr = optimizer.param_groups[0]['lr']

        print(f"{phase.capitalize()} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, AUC: {epoch_auc:.4f}, LR: {current_lr:.6f}")

        if phase == 'val':
            scheduler.step(epoch_auc)
            if epoch_auc > best_val_auc:
                best_val_auc = epoch_auc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), "best_model.pth")
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print("Early stopping triggered")
        break


Epoch 1/20


Train - Loss: 0.1036, Accuracy: 0.9596, AUC: 0.9932, LR: 0.001000


Val - Loss: 0.1101, Accuracy: 0.9551, AUC: 0.9945, LR: 0.001000

Epoch 2/20


Train - Loss: 0.0640, Accuracy: 0.9747, AUC: 0.9973, LR: 0.001000


Val - Loss: 0.1076, Accuracy: 0.9577, AUC: 0.9934, LR: 0.001000

Epoch 3/20


...

Val - Loss: 0.0769, Accuracy: 0.9729, AUC: 0.9974, LR: 0.001000

Epoch 8/20


Train - Loss: 0.0205, Accuracy: 0.9919, AUC: 0.9997, LR: 0.001000


Val - Loss: 0.0755, Accuracy: 0.9768, AUC: 0.9973, LR: 0.001000

Epoch 9/20


Train - Loss: 0.0186, Accuracy: 0.9925, AUC: 0.9998, LR: 0.001000


Val - Loss: 0.0963, Accuracy: 0.9721, AUC: 0.9973, LR: 0.001000
Early stopping triggered


In [15]:
model.eval()
test_loss = 0.0
test_labels = []
test_preds = []

pbar_test = tqdm(dataloaders['test'], desc="Testing Phase", leave=False)
with torch.no_grad():
    for inputs, labels in pbar_test:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        preds = torch.sigmoid(outputs)
        test_preds.extend(preds.cpu().numpy().flatten().tolist())
        test_labels.extend(labels.cpu().numpy().flatten().tolist())
        pbar_test.set_postfix({"Loss": f"{loss.item():.4f}"})

In [16]:
test_loss = test_loss / len(test_dataset)
try:
    test_auc = roc_auc_score(test_labels, test_preds)
except ValueError:
    test_auc = 0.0

test_preds_labels = (np.array(test_preds) >= 0.5).astype(int)
test_labels_np = np.array(test_labels).astype(int)
test_accuracy = (test_preds_labels == test_labels_np).mean()
current_lr = optimizer.param_groups[0]['lr']

print("\nTest Metrics:")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test AUC: {test_auc:.4f}, LR: {current_lr:.6f}")


Test Metrics:
Test Loss: 0.2609, Test Accuracy: 0.9265, Test AUC: 0.9816, LR: 0.000500


In [17]:
# Load best model weights
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [18]:
model.eval()
test_loss = 0.0
test_labels = []
test_preds = []

pbar_test = tqdm(dataloaders['test'], desc="Testing Phase", leave=False)
with torch.no_grad():
    for inputs, labels in pbar_test:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        preds = torch.sigmoid(outputs)
        test_preds.extend(preds.cpu().numpy().flatten().tolist())
        test_labels.extend(labels.cpu().numpy().flatten().tolist())
        pbar_test.set_postfix({"Loss": f"{loss.item():.4f}"})

In [19]:
test_loss = test_loss / len(test_dataset)
try:
    test_auc = roc_auc_score(test_labels, test_preds)
except ValueError:
    test_auc = 0.0

test_preds_labels = (np.array(test_preds) >= 0.5).astype(int)
test_labels_np = np.array(test_labels).astype(int)
test_accuracy = (test_preds_labels == test_labels_np).mean()
current_lr = optimizer.param_groups[0]['lr']

print("\nTest Metrics:")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test AUC: {test_auc:.4f}, LR: {current_lr:.6f}")


Test Metrics:
Test Loss: 0.2227, Test Accuracy: 0.9278, Test AUC: 0.9843, LR: 0.000500
